# Neural network from scratch

## Import libraries and loading the MNIST dataset

In this notebook, I will implement a simple neural network with two layers - a hidden layer with 16 nodes and an output layer with 10 nodes (as we are classifying numbers from *0-9*).

In [8]:
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import struct
from array import array

class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)
    
input_path = 'data'
training_images_filepath = r"data\train-images-idx3-ubyte\train-images-idx3-ubyte"
training_labels_filepath = r"data\train-labels-idx1-ubyte\train-labels-idx1-ubyte"
test_images_filepath = r"data\t10k-images-idx3-ubyte\t10k-images-idx3-ubyte"
test_labels_filepath = r"data\t10k-labels-idx1-ubyte\t10k-labels-idx1-ubyte"

mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

# Convert to numpy arrays and flatten the 28x28 images
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

x_train = x_train.reshape(x_train.shape[0], -1).transpose()
y_train = y_train.reshape(y_train.shape[0], -1).transpose()
x_test = x_test.reshape(x_test.shape[0], -1).transpose()
y_test = y_test.reshape(y_test.shape[0], -1).transpose()
print("Dimensions of x_train:", x_train.shape)
print("Dimensions of y_train:", y_train.shape)
print("Dimensions of x_test:", x_test.shape)
print("Dimensions of y_test:", y_test.shape)

_, m = x_train.shape

Dimensions of x_train: (784, 60000)
Dimensions of y_train: (1, 60000)
Dimensions of x_test: (784, 10000)
Dimensions of y_test: (1, 10000)


There are 60000, $28\times28$ pixel images in the training dataset which have been flattened to $784$ pixels. *x_train* has dimensions of $(784\times60000)$.
*y_train* has dimensions of $60000\times1$, with the columns containing the class label for the images in *x_train*.

## Forward propogation

The equations for the first  hidden layer,

\begin{align*}
Z^{[1]} &= W^{[1]}\cdot X + b^{[1]} \\
A^{[1]} &= g_{ReLU}(Z^{[1]})
\end{align*}

The equations for the second layer or the output layer,

\begin{align*}
Z^{[2]} &= W^{[2]}\cdot A^{[1]} + b^{[2]} \\
A^{[2]} &= g_{softmax}(Z^{[2]})
\end{align*}

where,\
\
$X$ or $A^{[0]}$ will be our *x_train* array with dimensions $784\times60000$ \
$W^{[1]}$ is the weight matrix of the first hidden layer and has dimensions $16\times784$, as there are 16 nodes and the dot product $(W^{[1]}\cdot X)$ results with a matrix of dimensions $16\times60000$ \
$b^{[1]}$ is is the bias matrix for the first hidden layer and has dimensions $16\times1$, broadcasting during matrix addition results in $Z^{[1]}$ with dimensions of $16\times60000$ \
\
Passing $Z^{[1]}$ through a softmax activation function results in $A^{[1]}$ with dimensions $16\times60000$, this will be the input to the output layer $Z^{[2]}$ \
The weight matrix for the output layer with 10 nodes for the 10 classes $W^{[2]}$ will have dimensions $10\times16$, as it is recieves 16 features from the first hidden layer. The resulting dot product of $(W^{[2]}\cdot A^{[1]})$ will end up having dimensions $10\times60000$ and the bias matrix for the output layer $b^{[2]}$ will have dimensions $10\times1$. \
\
The resulting $Z^{[2]}$ will have dimensions $10\times60000$ which is then passed through a softmax function to get the probabilities of each class (our output). 


### Code


In [2]:
# The weight matrices are initialized randomly and the bias matrices are initialized as zero for the first pass 
def initialize_params(input_layer_nodes=784,hidden_layer_nodes=16,output_layer_nodes=10):
    W1 = np.random.rand(hidden_layer_nodes, input_layer_nodes) * 0.1
    b1 = np.zeros((hidden_layer_nodes, 1))
    W2 = np.random.rand(output_layer_nodes, hidden_layer_nodes) * 0.1
    b2 = np.zeros((output_layer_nodes, 1))
    return W1, b1, W2, b2
# Rectified linear activation function
def ReLU(Z):
    A = np.maximum(Z, 0)
    return A

# Softmax activation function
def Softmax(Z):
    A = np.exp(Z)/sum(np.exp(Z))
    return A
# Forward propogation
def forward(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 =W2.dot(A1) + b2
    A2 = Softmax(Z2)
    return Z1, A1, Z2, A2

## Backward propogation

Our goal is to find parameters of $W$ and $b$ such that we minimize the cross-entropy loss function $L$ given by, 


$$L = - \sum_{i}Y_{i}log(A_{i}^{[2]})$$

where $Y_{i}$ is one-hot encoded. We do this using gradient descent - the gradient of the loss in the output layer:

$$\frac{\partial L}{\partial Z^{[2]}} = \frac{\partial L}{\partial A^{[2]}} \cdot \frac{\partial A^{[2]}}{\partial Z^{[2]}} $$

this simplifies to

$$dZ^{[2]} = \frac{\partial L}{\partial Z^{[2]}} = A^{[2]} - Y$$

The gradient of the loss in the output layer weights is,

$$dW^{[2]} = \frac{\partial L}{\partial W^{[2]}} = \frac{\partial L}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial W^{[2]}}$$

which becomes:

$$dW^{[2]} = \frac{1}{m}dZ^{[2]}A^{[1]T} $$

as $$Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$$

$$db^{[2]} = \frac{1}{m}\sum dZ^{[2]}$$

Similarly for the hidden layer,
$$dZ^{[1]} = W^{[2]T}dZ^{[2]} \cdot g^{[1]'}Z^{[1]}$$

$$dW^{[1]} = \frac{1}{m}dZ^{[1]}A^{[0]T}$$

$$db^{[1]} = \frac{1}{m}\sum dZ{[1]}$$

where $g^{[1]'}$ is the derivative of the ReLU function.

### Code

In [3]:
# Function for one hot encoding
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

# Function for the derivative of the ReLU function
def derivative_ReLU(Z):
    return Z>0

# Backward propogation
def backward(Z1, A1, Z2, A2, W1, W2, X, Y):
    dZ2 = A2 - one_hot(Y)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * derivative_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

## Update parameters

$$W^{[2]}_{new} = W{[2]} - \alpha dW^{[2]}$$

$$b^{[2]}_{new} = b^{[2]} - \alpha db^{[2]}$$

$$W^{[1]}_{new} = W{[1]} - \alpha dW^{[1]}$$

$$b^{[1]}_{new} = b^{[1]} - \alpha db^{[1]}$$

where $\alpha$ is the learning rate.

### Code

In [4]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    return W1, b1, W2, b2

## Putting it all together

In [13]:
# Function to print logs
def log_box(iteration, accuracy):
    iteration_str = f"Iteration: {iteration}"
    accuracy_str = f"Accuracy: {accuracy:.2f}"
    max_length = max(len(iteration_str), len(accuracy_str))

    pad_left = (max_length - len(iteration_str)) // 2
    pad_right = max_length - len(iteration_str) - pad_left

    border = '#' * (max_length + 4)
    print(border)
    print('#' + ' ' * (pad_left + 1) + iteration_str + ' ' * (pad_right + 1) + '#')
    print('# ' + accuracy_str + ' ' * (max_length - len(accuracy_str)) + ' #')
    print(border)



# Functions to calculate accuracy
def get_accuracy(Y_hat, Y):
    return np.sum(Y_hat == Y) / Y.size

def learn(X, Y, learning_rate, steps):
    W1, b1, W2, b2 = initialize_params()
    for i in range(steps):
        Z1, A1, Z2, A2 = forward(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
        # Log accuracy every 10 steps
        if i % 50 == 0:
            Y_hat = np.argmax(A2, 0)
            accuracy = get_accuracy(Y_hat, Y)
            log_box(i, accuracy)
    return W1, b1, W2, b2

In [14]:
# Train the model
learning_rate = 0.1
steps = 500
W1, b1, W2, b2 = learn(x_train/255, y_train, learning_rate, steps)

##################
#  Iteration: 0  #
# Accuracy: 0.10 #
##################
##################
# Iteration: 50  #
# Accuracy: 0.57 #
##################
##################
# Iteration: 100 #
# Accuracy: 0.80 #
##################
##################
# Iteration: 150 #
# Accuracy: 0.84 #
##################
##################
# Iteration: 200 #
# Accuracy: 0.86 #
##################
##################
# Iteration: 250 #
# Accuracy: 0.87 #
##################
##################
# Iteration: 300 #
# Accuracy: 0.88 #
##################
##################
# Iteration: 350 #
# Accuracy: 0.89 #
##################
##################
# Iteration: 400 #
# Accuracy: 0.89 #
##################
##################
# Iteration: 450 #
# Accuracy: 0.90 #
##################
